# Добавление для данных нового признака - прямого расстояния от дома до эспланады (центра Перми)

In [25]:
import pandas as pd
import numpy as np
import re

## Загружаем исходные данные

In [26]:
df = pd.read_csv('processed_data.csv', sep=';')

In [27]:
df.head()

,number of rooms,area of apartment,number of floors,apartment floor,price,repair,bathroom,terrace,year of construction,elevator,garbage chute,type of house,district,street,concierge
0,3,61.0,5,1,3200000.0,косметический,раздельный,балкон,1972,0,0,панельный,р-н Индустриальный,"улица Архитектора Свиязева, 50/2",0
1,1,31.0,2,1,1500000.0,косметический,совмещенный,нет,1957,0,0,кирпичный,р-н Свердловский,"улица Кузбасская, 41",0
2,3,89.0,25,3,6990000.0,евро,раздельный,лоджия,2010,2,0,монолитный,р-н Индустриальный,"шоссе Космонавтов, 215",1
3,3,84.0,6,6,6300000.0,косметический,несколько,балкон,2016,1,0,кирпичный,р-н Мотовилихинский,"улица КИМ, 74А",1
4,3,64.0,10,6,4350000.0,косметический,раздельный,балкон,1993,1,1,панельный,р-н Индустриальный,"улица Норильская, 9",0


## Функция перевода градусов в длину

In [28]:
import math

def get_dist(llong1, llat1, llong2, llat2):
    
    rad = 6372795
    
    #в радианах
    lat1 = llat1*math.pi/180.
    lat2 = llat2*math.pi/180.
    long1 = llong1*math.pi/180.
    long2 = llong2*math.pi/180.
    
    #косинусы и синусы широт и разницы долгот
    cl1 = math.cos(lat1)
    cl2 = math.cos(lat2)
    sl1 = math.sin(lat1)
    sl2 = math.sin(lat2)
    delta = long2 - long1
    cdelta = math.cos(delta)
    sdelta = math.sin(delta)
    
    #вычисления длины большого круга
    y = math.sqrt(math.pow(cl2*sdelta,2)+math.pow(cl1*sl2-sl1*cl2*cdelta,2))
    x = sl1*sl2+cl1*cl2*cdelta
    ad = math.atan2(y,x)
    dist = ad*rad
    
    return dist

In [23]:
dists = np.zeros(len(df))
esplanada_long = 56.224395
esplanada_lat =  58.009535

## Бот для обращения к сайту-геокодеру

In [29]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time

driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\Daniil\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


In [33]:
for i in range(146, len(df)):
    driver.get("https://geotree.ru/geocoder")
    time.sleep(1)
    elem = driver.find_element_by_xpath("/html/body/div[1]/div/div/main/div[4]/input[2]")
    elem.clear()
    time.sleep(1)
    elem.send_keys("Пермский край, город Пермь, " + df['street'].values[i])
    time.sleep(1)
    elem.send_keys(" ")
    time.sleep(1)
    json = driver.find_element_by_xpath("/html/body/div[1]/div/div/main/div[6]/pre")
    if re.search("geo_center", json.text) is None:
        print(f"Error with {i}" + " Text: " +  df['street'].values[i])
        time.sleep(2)
        continue
    begin_index = re.search("geo_center", json.text).span()[1] + 24
    end_index = re.search("geo_inside", json.text).span()[0] - 21
    try:
        long = float(json.text[begin_index: begin_index+7])
        lat = float(json.text[end_index-9: end_index])
    except ValueError:
        print(f"Error with {i}" + " Dist")
        time.sleep(2)
        continue
    dist = get_dist(esplanada_long, esplanada_lat, long, lat)
    print(f"Success with {i} Dist:" + str(dist))
    print("-----------------------")
    dists[i] = dist
    time.sleep(3)

Error with 146 Dist
Success with 147 Dist:2922.7241569110356
-----------------------
Success with 148 Dist:2431.8052097233094
-----------------------
Success with 149 Dist:4277.0491137380295
-----------------------
Success with 150 Dist:2456.217659057953
-----------------------
Success with 151 Dist:5952.517048409784
-----------------------
Success with 152 Dist:2936.9654243459004
-----------------------
Success with 153 Dist:4745.06519972934
-----------------------
Success with 154 Dist:2936.9654243459004
-----------------------
Success with 155 Dist:4745.06519972934
-----------------------
Success with 156 Dist:22629.684884137558
-----------------------
Success with 157 Dist:8865.898783166675
-----------------------
Success with 158 Dist:11666.124063279121
-----------------------
Success with 159 Dist:1261.686871699992
-----------------------
Success with 160 Dist:14706.491740177562
-----------------------
Success with 161 Dist:16193.817623995907
-----------------------
Success with 

Success with 276 Dist:2530.719603606261
-----------------------
Success with 277 Dist:1663.5754695075775
-----------------------
Success with 278 Dist:1165.0528057171587
-----------------------
Success with 279 Dist:3168.5982509048795
-----------------------
Success with 280 Dist:9223.468777999098
-----------------------
Success with 281 Dist:2372.2951021592194
-----------------------
Success with 282 Dist:1215.413284087935
-----------------------
Success with 283 Dist:3387.6630011394573
-----------------------
Success with 284 Dist:4916.533521439176
-----------------------
Error with 285 Text: улица Кузбасская, 35
Success with 286 Dist:4141.901651019999
-----------------------
Success with 287 Dist:2151.195532657802
-----------------------
Success with 288 Dist:11438.17466002151
-----------------------
Success with 289 Dist:4021.5109290551436
-----------------------
Success with 290 Dist:5213.860697378696
-----------------------
Error with 291 Text: улица Свободы, 21А
Success with 292

Success with 406 Dist:6306.644045346485
-----------------------
Success with 407 Dist:6792.482129533168
-----------------------
Success with 408 Dist:4410.902690378007
-----------------------
Success with 409 Dist:1162.1937629195504
-----------------------
Success with 410 Dist:6460.673319485657
-----------------------
Success with 411 Dist:323.6167662557067
-----------------------
Success with 412 Dist:6885.404926993601
-----------------------
Success with 413 Dist:3677.8914511565717
-----------------------
Success with 414 Dist:2139.4557997097163
-----------------------
Success with 415 Dist:14343.270846043879
-----------------------
Success with 416 Dist:1780.254521944828
-----------------------
Success with 417 Dist:2661.15187533336
-----------------------
Success with 418 Dist:11905.623082225602
-----------------------
Success with 419 Dist:4500.014820208134
-----------------------
Success with 420 Dist:1732.6416718978935
-----------------------
Success with 421 Dist:2237.07866106

Success with 536 Dist:14147.69511291561
-----------------------
Error with 537 Dist
Success with 538 Dist:12103.722673129905
-----------------------
Success with 539 Dist:11115.161035619973
-----------------------
Success with 540 Dist:11483.580260712086
-----------------------
Success with 541 Dist:2386.224696932995
-----------------------
Success with 542 Dist:3176.4750090348557
-----------------------
Success with 543 Dist:5476.396022783006
-----------------------
Error with 544 Text: улица Карбышева, 78/2
Success with 545 Dist:3378.60974227904
-----------------------
Success with 546 Dist:9501.71221057005
-----------------------
Success with 547 Dist:2287.719882966373
-----------------------
Success with 548 Dist:1250.3187885742968
-----------------------
Success with 549 Dist:9179.908811132753
-----------------------
Success with 550 Dist:10202.211186743036
-----------------------
Success with 551 Dist:5145.142717057842
-----------------------
Success with 552 Dist:11596.461195477

Success with 669 Dist:15533.179050467686
-----------------------
Success with 670 Dist:2814.7818377203175
-----------------------
Success with 671 Dist:12627.932199376513
-----------------------
Success with 672 Dist:13609.387611599
-----------------------
Success with 673 Dist:4476.778960354897
-----------------------
Success with 674 Dist:2674.129868264719
-----------------------
Success with 675 Dist:5507.441968506241
-----------------------
Success with 676 Dist:1306.1483163342089
-----------------------
Success with 677 Dist:3390.3159385674862
-----------------------
Success with 678 Dist:22415.06519267355
-----------------------
Success with 679 Dist:2948.684867453179
-----------------------
Success with 680 Dist:9190.05876184996
-----------------------
Success with 681 Dist:7761.077707539435
-----------------------
Success with 682 Dist:23272.97199566279
-----------------------
Error with 683 Text: улица Овчинникова, 15А
Success with 684 Dist:17039.60291019782
------------------

Success with 800 Dist:9407.4626493306
-----------------------
Success with 801 Dist:5799.935897546818
-----------------------
Success with 802 Dist:2820.081421737791
-----------------------
Success with 803 Dist:3699.2097615633243
-----------------------
Success with 804 Dist:4462.158492606881
-----------------------
Success with 805 Dist:3936.1999337167495
-----------------------
Success with 806 Dist:5029.106954689874
-----------------------
Success with 807 Dist:778.0535084071236
-----------------------
Success with 808 Dist:15053.779569830003
-----------------------
Success with 809 Dist:1784.1810213112844
-----------------------
Success with 810 Dist:2289.4539457430315
-----------------------
Success with 811 Dist:6876.200359469137
-----------------------
Success with 812 Dist:4688.101784619709
-----------------------
Success with 813 Dist:882.4407962094826
-----------------------
Success with 814 Dist:5187.51233695628
-----------------------
Success with 815 Dist:9277.15090689804

Success with 932 Dist:21057.744642499718
-----------------------
Success with 933 Dist:4722.991694229498
-----------------------
Success with 934 Dist:448.856121985166
-----------------------
Success with 935 Dist:1405.4793639319914
-----------------------
Success with 936 Dist:5629.157696940996
-----------------------
Error with 937 Text: улица Гашкова, 28
Success with 938 Dist:3180.7438717494306
-----------------------
Success with 939 Dist:3105.7089260232488
-----------------------
Success with 940 Dist:5521.661083185583
-----------------------
Error with 941 Text: улица ская улица, 5
Success with 942 Dist:3820.3178118556734
-----------------------
Error with 943 Text: улица Евгения Пермяка, 1
Success with 944 Dist:3042.066289689201
-----------------------
Success with 945 Dist:4895.535888905948
-----------------------
Success with 946 Dist:23149.61347939299
-----------------------
Success with 947 Dist:4779.8715106632835
-----------------------
Success with 948 Dist:5022.2253262742

Success with 1062 Dist:5213.860697378696
-----------------------
Success with 1063 Dist:5426.636963280267
-----------------------
Success with 1064 Dist:22258.62651669251
-----------------------
Success with 1065 Dist:5159.190459844357
-----------------------
Error with 1066 Text: улица Лодыгина, 46к2
Error with 1067 Text: улица Мензелинская, 14
Success with 1068 Dist:4308.730340938217
-----------------------
Success with 1069 Dist:7320.348915402611
-----------------------
Success with 1070 Dist:9223.468777999098
-----------------------
Success with 1071 Dist:12309.703079000761
-----------------------
Success with 1072 Dist:2075.2287867976693
-----------------------
Success with 1073 Dist:8865.898783166675
-----------------------
Success with 1074 Dist:2922.7241569110356
-----------------------
Success with 1075 Dist:2913.807381287377
-----------------------
Success with 1076 Dist:17105.225447751644
-----------------------
Success with 1077 Dist:7938.288209444231
----------------------

## Ручное дозаполнение

In [48]:
for i in range(len(dists)):
    if dists[i] == 0:
        print(i)
        print("Пермь " + df['street'][i])
        data = input().split(', ')
        long = float(data[1])
        lat = float(data[0])
        dists[i] = get_dist(esplanada_long, esplanada_lat, long, lat)
        print(dists[i])
        print("-----")

1054
Пермь улица Евгения Пермяка, 1
58.054831, 56.351839
9039.148459223397
-----
1055
Пермь улица Делегатская, 39Б
58.135157, 56.399450
17356.800723893994
-----
1058
Пермь улица посёлок, Ляды,, ул.,
0, 0
8105156.956837158
-----
1066
Пермь улица Лодыгина, 46к2
57.965808, 56.217074
4882.708093086754
-----
1067
Пермь улица Мензелинская, 14
58.030441, 55.846465
22384.18497556184
-----
1089
Пермь улица Крайпрудская, 44
58.036326, 56.330845
6942.299413220976
-----
1104
Пермь улица 40-летия Победы, 16
58.047992, 56.582427
21515.239032913265
-----
1112
Пермь улица Мира, 100
57.974396, 56.170801
5025.776661375873
-----
1114
Пермь улица Красноводская, 18
58.046797, 56.082371
9334.924162925734
-----


## Некоторые некорректные значения

In [51]:
for i in range(len(dists)):
    if dists[i] == get_dist(esplanada_long, esplanada_lat, 0, 0):
        print(i)
        print("Пермь " + df['street'][i])
        dists[i] = None

759
Пермь улица кая улица, 9
941
Пермь улица ская улица, 5
1058
Пермь улица посёлок, Ляды,, ул.,


## Сохраняем данные

In [52]:
df['distance'] = dists

In [55]:
df = df[df['distance'].isna() == False]

In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1130 entries, 0 to 1132
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   number of rooms       1130 non-null   int64  
 1   area of apartment     1130 non-null   float64
 2   number of floors      1130 non-null   int64  
 3   apartment floor       1130 non-null   int64  
 4   price                 1130 non-null   float64
 5   repair                1130 non-null   object 
 6   bathroom              1130 non-null   object 
 7   terrace               1130 non-null   object 
 8   year of construction  1130 non-null   int64  
 9   elevator              1130 non-null   int64  
 10  garbage chute         1130 non-null   int64  
 11  type of house         1130 non-null   object 
 12  district              1130 non-null   object 
 13  street                1130 non-null   object 
 14  concierge             1130 non-null   int64  
 15  distance             

In [59]:
df.to_csv('data_with_dists.csv', sep=';', index=False)